# Train YOLOv8m for Traffic Sign Detection (Single Class)

This notebook is designed to train a **YOLOv8m** model specifically for **traffic sign detection** on Kaggle.

**Changes for Accuracy:**
- **Model:** Switched to YOLOv8m (Medium) for a better balance of accuracy and generalization compared to the larger models which were overfitting.
- **Single Class Training:** We are training with `single_cls=True`. This treats all traffic sign types as a single "Traffic Sign" class. This significantly improves detection performance (recall/precision) because the model focuses solely on distinguishing signs from background (houses, trees, etc.) without getting confused by class variations. Classification is handled by a separate model in the pipeline.

**Prerequisites:**
1.  **Datasets:** Ensure you have added your datasets (DFG, BelgiumTSC) to the Kaggle notebook. They should appear under `/kaggle/input`.
2.  **GPU:** Make sure to turn on the GPU accelerator in the Kaggle notebook settings.


In [23]:
!pip uninstall -y opencv-python opencv-python-headless opencv-contrib-python
!pip uninstall -y ultralytics


Found existing installation: opencv-python 4.12.0.88
Uninstalling opencv-python-4.12.0.88:
  Successfully uninstalled opencv-python-4.12.0.88
Found existing installation: opencv-python-headless 4.10.0.84
Uninstalling opencv-python-headless-4.10.0.84:
  Successfully uninstalled opencv-python-headless-4.10.0.84
Found existing installation: ultralytics 8.3.232
Uninstalling ultralytics-8.3.232:
  Successfully uninstalled ultralytics-8.3.232


In [24]:
!pip install -U ultralytics
!pip install -qU "numpy<2.0" "opencv-python-headless==4.10.*"

  Using cached ultralytics-8.3.232-py3-none-any.whl.metadata (37 kB)
  Using cached opencv_python-4.12.0.88-cp37-abi3-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (19 kB)
  Using cached numpy-2.2.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached ultralytics-8.3.232-py3-none-any.whl (1.1 MB)
Using cached opencv_python-4.12.0.88-cp37-abi3-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (67.0 MB)
Using cached numpy-2.2.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
easyocr 1.7.2 requires opencv-python-headless, which is not installed.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0

## Prepare Datasets

We will run the `prepare_datasets.py` script. 
**IMPORTANT:** Before running this, verify that the paths in `prepare_datasets.py` match your actual Kaggle input paths.
You can list the input directory to check:

In [ ]:
# List input datasets to verify names
!ls /kaggle/input

In [31]:
import os
import json
import csv
import shutil
import yaml
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from PIL import Image
import cv2
import numpy as np

# Configuration - Kaggle Environment
DATASET_ROOT = Path("/kaggle/input")
OUTPUT_ROOT = Path("/kaggle/working/traffic_signs_yolo")

# Clean up previous run to avoid duplicate labels
if OUTPUT_ROOT.exists():
    shutil.rmtree(OUTPUT_ROOT)

# Paths to specific datasets
BELGIUM_PATH = DATASET_ROOT / "belgium-ts-dataset"
NEW_DATASET_PATH = DATASET_ROOT / "traffic-sign-detection-dataset"

# Create directories
IMAGES_TRAIN = OUTPUT_ROOT / "images/train"
IMAGES_VAL = OUTPUT_ROOT / "images/val"
LABELS_TRAIN = OUTPUT_ROOT / "labels/train"
LABELS_VAL = OUTPUT_ROOT / "labels/val"

for p in [IMAGES_TRAIN, IMAGES_VAL, LABELS_TRAIN, LABELS_VAL]:
    p.mkdir(parents=True, exist_ok=True)

def convert_bbox_to_yolo(size, box):
    dw = 1. / size[0]
    dh = 1. / size[1]
    x = (box[0] + box[2]) / 2.0
    y = (box[1] + box[3]) / 2.0
    w = box[2] - box[0]
    h = box[3] - box[1]
    return (x * dw, y * dh, w * dw, h * dh)

def find_file_recursive(root_dir, filename_pattern):
    """Finds a file matching pattern recursively."""
    matches = list(root_dir.rglob(filename_pattern))
    if matches:
        return matches[0]
    return None

def find_dir_recursive(root_dir, dirname):
    """Finds a directory matching name recursively."""
    for path in root_dir.rglob(dirname):
        if path.is_dir():
            return path
    return None

def process_belgium(root_dir):
    print("Processing BelgiumTSC dataset...")
    
    training_gt_file = find_file_recursive(root_dir, "training_GT.txt")
    if not training_gt_file:
        print("Could not find training_GT.txt")
        return

    print(f"Reading annotations from {training_gt_file}")
    with open(training_gt_file, 'r') as f:
        lines = [l.strip() for l in f.readlines() if l.strip()]

    # Pre-scan all images to map ID -> Path
    # Filename format: image.XXXXXX.jp2
    print("Scanning for images...")
    image_map = {} # "000508" -> Path
    for ext in ['*.jp2', '*.jpg', '*.png']:
        for img_p in root_dir.rglob(ext):
            # Extract ID from "image.000508.jp2" -> "000508"
            parts = img_p.name.split('.')
            if len(parts) >= 2 and parts[0] == 'image':
                img_id_str = parts[1] # "000508"
                # Store both with and without leading zeros to be safe
                image_map[img_id_str] = img_p
                image_map[str(int(img_id_str))] = img_p
    
    print(f"Found {len(image_map)} images.")

    i = 0
    count = 0
    while i < len(lines):
        line = lines[i]
        
        # Heuristic to find the start of a block:
        # Look for the class definition line: e.g., "1;C43;" or "2;X21;B15A;"
        # It starts with a number and contains semicolons.
        if ';' in line and line[0].isdigit():
            # Found Header Class Line
            try:
                # Next line is Num Images
                i += 1
                num_images = int(lines[i])
                
                # Read Image IDs
                seq_images = [] # List of (ImageID, Path)
                for _ in range(num_images):
                    i += 1
                    parts = lines[i].split()
                    img_id = parts[-1] # Last number is ID
                    seq_images.append(img_id)
                
                # Next line is Num Annotations
                i += 1
                num_annotations = int(lines[i])
                
                # Read Annotations
                for _ in range(num_annotations):
                    # Line 1: Coords x1 y1 x2 y2
                    i += 1
                    coords = list(map(float, lines[i].split()))
                    x1, y1, x2, y2 = coords
                    
                    # Line 2: Meta ? ? ImageIndex(1-based)
                    i += 1
                    meta = list(map(float, lines[i].split()))
                    img_idx_1based = int(meta[-1])
                    
                    # Line 3: Class
                    i += 1
                    # class_name = lines[i].replace(';', '')
                    
                    # Line 4: 3D Coords (Skip)
                    i += 1
                    
                    # Process this annotation
                    if 1 <= img_idx_1based <= len(seq_images):
                        img_id = seq_images[img_idx_1based - 1]
                        
                        if img_id in image_map:
                            img_path = image_map[img_id]
                            
                            # Load image
                            try:
                                img = cv2.imread(str(img_path))
                                if img is None:
                                    pil_img = Image.open(img_path)
                                    img = np.array(pil_img)
                                    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
                                
                                h, w = img.shape[:2]
                                
                                # Save Image
                                new_filename = f"belgium_{img_path.parent.name}_{img_path.stem}.jpg"
                                is_train = hash(new_filename) % 10 < 8
                                target_img_dir = IMAGES_TRAIN if is_train else IMAGES_VAL
                                target_lbl_dir = LABELS_TRAIN if is_train else LABELS_VAL
                                
                                save_path = target_img_dir / new_filename
                                if not save_path.exists():
                                    cv2.imwrite(str(save_path), img)
                                
                                # Save Label
                                yolo_box = convert_bbox_to_yolo((w, h), (x1, y1, x2, y2))
                                label_file = target_lbl_dir / new_filename.replace('.jpg', '.txt')
                                
                                with open(label_file, 'a') as lf:
                                    lf.write(f"0 {yolo_box[0]:.6f} {yolo_box[1]:.6f} {yolo_box[2]:.6f} {yolo_box[3]:.6f}\n")
                                count += 1
                                
                            except Exception as e:
                                # print(f"Error: {e}")
                                pass
            except (ValueError, IndexError) as e:
                # print(f"Parsing error at line {i}: {e}")
                pass
        
        i += 1

    print(f"Processed {count} annotations for BelgiumTSC.")

def process_new_dataset(root_dir):
    print(f"Processing new dataset from {root_dir}...")
    
    images_root = find_dir_recursive(root_dir, "images")
    labels_root = find_dir_recursive(root_dir, "labels")
    
    if not images_root or not labels_root:
        print("Could not find 'images' or 'labels' directories.")
        return

    for split in ['train', 'val']:
        src_img_dir = images_root / split
        src_lbl_dir = labels_root / split
        
        if not src_img_dir.exists():
            # Try to find if train/val are inside another folder or flat
            # If not found, just skip
            continue
            
        # Target dirs
        dst_img_dir = IMAGES_TRAIN if split == 'train' else IMAGES_VAL
        dst_lbl_dir = LABELS_TRAIN if split == 'train' else LABELS_VAL
        
        count = 0
        for img_path in src_img_dir.glob("*"):
            if img_path.suffix.lower() not in ['.jpg', '.jpeg', '.png', '.bmp']:
                continue
                
            new_filename = f"new_ds_{split}_{img_path.name}"
            shutil.copy(img_path, dst_img_dir / new_filename)
            
            lbl_name = img_path.stem + ".txt"
            src_lbl_path = src_lbl_dir / lbl_name
            
            if src_lbl_path.exists():
                dst_lbl_path = dst_lbl_dir / (Path(new_filename).stem + ".txt")
                with open(src_lbl_path, 'r') as f_in, open(dst_lbl_path, 'w') as f_out:
                    for line in f_in:
                        parts = line.strip().split()
                        if len(parts) >= 5:
                            # Force class 0
                            new_line = f"0 {' '.join(parts[1:])}\n"
                            f_out.write(new_line)
                count += 1
        print(f"  Copied {count} images/labels from {split}.")

def create_yaml():
    yaml_content = {
        'path': str(OUTPUT_ROOT.absolute()),
        'train': 'images/train',
        'val': 'images/val',
        'names': {0: 'traffic_sign'}
    }
    with open(OUTPUT_ROOT / "data.yaml", 'w') as f:
        yaml.dump(yaml_content, f)
    print("Created data.yaml with 1 class: traffic_sign")

# Run
process_belgium(BELGIUM_PATH)
process_new_dataset(NEW_DATASET_PATH)
create_yaml()

Processing BelgiumTSC dataset...
Reading annotations from /kaggle/input/belgium-ts-dataset/annotations/training_GT.txt
Scanning for images...
Found 18012 images.
Processed 1599 annotations for BelgiumTSC.
Processing new dataset from /kaggle/input/traffic-sign-detection-dataset...
  Copied 7643 images/labels from train.
  Copied 1891 images/labels from val.
Created data.yaml with 1 class: traffic_sign


In [29]:
rm -rf /kaggle/working/runs

In [30]:
rm -rf /kaggle/working/traffic_signs_yolo

## Train Model

Now we train the YOLOv8m model. 
We use `single_cls=True` to merge all classes into one. This is the most effective "trick" to improve pure detection performance when you have a separate classifier.


In [ ]:
from ultralytics import YOLO
import os
from pathlib import Path
import torch
import gc

def train_model():
    # Set environment variable to help with memory fragmentation
    os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
    
    # Clear any existing memory
    gc.collect()
    torch.cuda.empty_cache()

    # Path to the data.yaml file generated by prepare_datasets.py
    data_yaml_path = Path("/kaggle/working/traffic_signs_yolo/data.yaml")
    
    if not data_yaml_path.exists():
        print(f"Error: {data_yaml_path} not found.")
        print("Please run the data preparation cell first to prepare your data.")
        return

    # Load a model
    try:
        model = YOLO('yolov8m.pt') 
    except Exception as e:
        print(f"Could not load yolov8m.pt. Error: {e}")
        model = YOLO('yolov8m.pt')

    # Train the model
    # SPEED FIX: 
    # - Reduced imgsz to 640 (Standard YOLO size). 1280 was 4x slower.
    # - Increased batch to 32 for better GPU utilization.
    results = model.train(
        data=str(data_yaml_path.absolute()),
        epochs=100,
        imgsz=640,   # <--- Changed from 1280 to 640 for speed
        batch=16,    # <--- Increased from 8 to 32
        name='traffic_sign_yolov8m_det',
        device=0,
        patience=20,
        save=True,
        exist_ok=True,
        single_cls=True, 
        close_mosaic=10,
        cos_lr=True,
        val=True
    )

    # Validate
    metrics = model.val()
    print(f"mAP50-95: {metrics.box.map}")

    # Export
    success = model.export(format='onnx')
    print(f"Model exported to ONNX: {success}")

# Run training
train_model()

Ultralytics 8.3.232 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/kaggle/working/traffic_signs_yolo/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=traffic_sign_yolov8m_det, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patien

In [33]:
rm -rf /kaggle/working/runs

## Visualize Results

After training, we can look at the results. YOLO saves results in `runs/detect/traffic_sign_yolo11x`.

In [ ]:
from IPython.display import Image
import glob
import os

# Find the latest run directory
runs = sorted(glob.glob('runs/detect/traffic_sign_yolov8m_det*'))
if runs:
    latest_run = runs[-1]
    print(f"Showing results from {latest_run}")
    
    # Display confusion matrix if exists
    cm_path = f"{latest_run}/confusion_matrix.png"
    if os.path.exists(cm_path):
        display(Image(filename=cm_path))
    
    # Display results.png
    results_path = f"{latest_run}/results.png"
    if os.path.exists(results_path):
        display(Image(filename=results_path))
else:
    print("No training results found.")

## Export Model

The best model is saved as `best.pt` in the weights folder. We should ensure it's in the output directory.

In [ ]:
# Copy best model to working directory root for easy download
import glob
import os

runs = sorted(glob.glob('runs/detect/traffic_sign_yolov8m_det*'))
if runs:
    latest_run = runs[-1]
    best_model = f"{latest_run}/weights/best.pt"
    
    if os.path.exists(best_model):
        !cp {best_model} /kaggle/working/best_yolov8m_det.pt
        print(f"Model copied to /kaggle/working/best_yolov8m_det.pt")
    else:
        print("best.pt not found")
else:
    print("No runs found")